<a href="https://colab.research.google.com/github/chousin/projek_vpn/blob/master/Tugas_RL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%writefile submission.py
import pickle
import bz2
import base64
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import time
from copy import deepcopy
from kaggle_environments.envs.hungry_geese.hungry_geese import Action, translate
from kaggle_environments.helpers import histogram

Writing submission.py


In [ ]:
%%writefile -a submission.py
class MCTS():
    def __init__(self, game, nn_agent, eps=1e-8, cpuct=1.0):
        self.game = game
        self.nn_agent = nn_agent
        self.eps = eps
        self.cpuct = cpuct
        self.gamma = 0.8

        self.Qsa = {}  # stores Q values for s,a (as defined in the paper)
        self.Nsa = {}  # stores #times edge s,a was visited
        self.Ns = {}  # stores #times board s was visited
        self.Ps = {}  # stores initial policy (returned by neural net)
        self.Vs = {}  # stores value for board s (returned by neural net)
        self.Ms = {}  # stores game.getValidMoves for board s

        self.last_obs = None
        self.rewards = [0] * 4

    def getActionProb(self, obs, timelimit=1.0):
        start_time = time.time()
        while time.time() - start_time < timelimit:
            self.search(obs, self.last_obs, self.rewards[:])
            
        for i in range(4):
            if len(obs.geese[i]) > 0:
                self.rewards[i] += 100
            elif self.last_obs is not None:
                self.rewards[i] += len(self.last_obs.geese[i])

        s = self.game.stringRepresentation(obs)
        i = obs.index
        counts = [
            self.Nsa[(s, i, a)] if (s, i, a) in self.Nsa else 0
            for a in range(self.game.getActionSize())
        ]
        print(counts)

        valids = np.array(self.game.getValidMoves(obs, self.last_obs, i))
        valid_counts = counts * valids
        sum_valid_counts = np.sum(valid_counts)
        if sum_valid_counts > 0:
            prob = valid_counts / sum_valid_counts
        else:
            valids = np.array(self.game.getNonOppositeMoves(obs, self.last_obs, i))
            valid_counts = counts * valids
            sum_valid_counts = np.sum(valid_counts)
            if sum_valid_counts > 0:
                prob = valid_counts / sum_valid_counts
            else:
                prob = [1.] * valids
                prob /= np.sum(prob)

        self.last_obs = obs
        return prob
    
    def get_values(self, obs, rewards):
        '''
        rank 1:    1
        rank 1.5:  0.666
        rank 2:   -1
        rank 2.5: -1
        rank 3:   -1
        rank 3.5: -1
        rank 4:   -1
        '''
        ranks = [1] * 4
        for i in range(4):
            for j in (j for j in range(4) if j != i):
                if rewards[i] < rewards[j]:
                    ranks[i] += 1
                elif rewards[i] == rewards[j]:
                    ranks[i] += 0.5
        values = [(5 - 2 * rank) / 3 if rank < 2 else -1 for rank in ranks]
        return values

    def search(self, obs, last_obs, rewards):
        s = self.game.stringRepresentation(obs)

        for i in range(4):
            if len(obs.geese[i]) > 0:
                rewards[i] += 100
            elif last_obs is not None:
                rewards[i] += len(last_obs.geese[i])

        if s not in self.Ns:
            values = self.get_values(obs, rewards)
#             print(values)

            for i in range(4):
                if len(obs.geese[i]) == 0 or obs.step == 199:
                    continue

                # leaf node
#                 if i == obs.index:
                self.Ps[(s, i)], values[i] = self.nn_agent.predict(obs, last_obs, i)
#                     print(values[i])
#                 else:
#                     self.Ps[(s, i)] = np.array([0.25, 0.25, 0.25, 0.25])

                valids = self.game.getValidMoves(obs, last_obs, i)
                self.Ps[(s, i)] = self.Ps[(s, i)] * valids  # masking invalid moves
                sum_Ps_s = np.sum(self.Ps[(s, i)])
                if sum_Ps_s > 0:
                    self.Ps[(s, i)] /= sum_Ps_s  # renormalize

                self.Vs[(s, i)] = values[i]
                self.Ms[(s, i)] = valids
                self.Ns[s] = 0

            return values

        best_acts = [None] * 4
        for i in range(4):
            if len(obs.geese[i]) == 0:
                continue

            # pick the action with the highest upper confidence bound
            valids = self.Ms[(s, i)]
            u = np.array([-float('inf')] * 4)
            for a in range(self.game.getActionSize()):
                if valids[a]:
                    if (s, i, a) in self.Qsa:
                        u[a] = self.Qsa[(s, i, a)] + self.cpuct * self.Ps[(s, i)][a] * math.sqrt(self.Ns[s]) / (1 + self.Nsa[(s, i, a)])
                    else:
                        u[a] = self.Vs[(s, i)] + self.cpuct * self.Ps[(s, i)][a] * math.sqrt(self.Ns[s] + self.eps)

            # random choice among the best actions
            best_acts[i] = self.game.actions[np.random.choice(np.argwhere(u == u.max()).flatten())]

        next_obs = self.game.getNextState(obs, last_obs, best_acts)
        values = self.search(next_obs, obs, rewards)

        for i in range(4):
            if len(obs.geese[i]) == 0:
                continue

            a = self.game.actions.index(best_acts[i])
            v = values[i]
            if (s, i, a) in self.Qsa:
                self.Qsa[(s, i, a)] = (self.Nsa[(s, i, a)] * self.Qsa[(s, i, a)] + v) / (self.Nsa[(s, i, a)] + 1)
                self.Nsa[(s, i, a)] += 1
            else:
                self.Qsa[(s, i, a)] = v
                self.Nsa[(s, i, a)] = 1

        self.Ns[s] += 1
        
        values = [value * self.gamma for value in values]
        return values

Appending to submission.py


In [ ]:
%%writefile -a submission.py
class HungryGeese(object):
    def __init__(self,
                 rows=7,
                 columns=11,
                 actions=[Action.NORTH, Action.SOUTH, Action.WEST, Action.EAST],
                 hunger_rate=40):
        self.rows = rows
        self.columns = columns
        self.actions = actions
        self.hunger_rate = hunger_rate

    def getActionSize(self):
        return len(self.actions)

    def getNextState(self, obs, last_obs, directions):
        next_obs = deepcopy(obs)
        next_obs.step += 1
        geese = next_obs.geese
        food = next_obs.food
        
        for i in range(4):
            goose = geese[i]
            
            if len(goose) == 0: 
                continue
            
            head = translate(goose[0], directions[i], self.columns, self.rows)

            # Check action direction
            if last_obs is not None and head == last_obs.geese[i][0]:
                geese[i] = []
                continue

            # Consume food or drop a tail piece.
            if head in food:
                food.remove(head)
            else:
                goose.pop()
            
            # Add New Head to the Goose.
            goose.insert(0, head)

            # If hunger strikes remove from the tail.
            if next_obs.step % self.hunger_rate == 0:
                if len(goose) > 0:
                    goose.pop()

        goose_positions = histogram(
            position
            for goose in geese
            for position in goose
        )

        # Check for collisions.
        for i in range(4):
            if len(geese[i]) > 0:
                head = geese[i][0]
                if goose_positions[head] > 1:
                    geese[i] = []
        
        return next_obs

    def getValidMoves(self, obs, last_obs, index):   
        geese = obs.geese
        pos = geese[index][0]
        obstacles = {position for goose in geese for position in goose[:-1]}
        if last_obs is not None: obstacles.add(last_obs.geese[index][0])
        
        valid_moves = [
            translate(pos, action, self.columns, self.rows) not in obstacles
            for action in self.actions
        ]

        return valid_moves
    
    def getNonOppositeMoves(self, obs, last_obs, index):   
        geese = obs.geese
        pos = geese[index][0]
        opposite = None
        if last_obs is not None: opposite = last_obs.geese[index][0]

        valid_moves = [
            translate(pos, action, self.columns, self.rows) != opposite
            for action in self.actions
        ]

        return valid_moves

    def stringRepresentation(self, obs):      
        return str(obs.geese + obs.food)

In [ ]:
%%writefile -a submission.py
# Neural Network for Hungry Geese
class TorusConv2d(nn.Module):
    def __init__(self, input_dim, output_dim, kernel_size, bn):
        super().__init__()
        self.edge_size = (kernel_size[0] // 2, kernel_size[1] // 2)
        self.conv = nn.Conv2d(input_dim, output_dim, kernel_size=kernel_size, padding=self.edge_size, padding_mode='circular')
        self.bn = nn.BatchNorm2d(output_dim) if bn else None

    def forward(self, x):
        h = self.conv(x)
        h = self.bn(h) if self.bn is not None else h
        return h


class GeeseNet(nn.Module):
    def __init__(self):
        super().__init__()
        layers, filters = 12, 32
        self.conv0 = TorusConv2d(17, filters, (3, 3), True)
        self.blocks = nn.ModuleList([TorusConv2d(filters, filters, (3, 3), True) for _ in range(layers)])
        self.head_p = nn.Linear(filters, 4, bias=False)
        self.head_v = nn.Linear(filters * 2, 1, bias=False)

    def forward(self, x):
        h = F.relu_(self.conv0(x))
        for block in self.blocks:
            h = F.relu_(h + block(h))
        h_head = (h * x[:,:1]).view(h.size(0), h.size(1), -1).sum(-1)
        h_avg = h.view(h.size(0), h.size(1), -1).mean(-1)
        p = torch.softmax(self.head_p(h_head), 1)
        v = torch.tanh(self.head_v(torch.cat([h_head, h_avg], 1)))

        return p, v


class NNAgent():
    def __init__(self, state_dict):
        self.model = GeeseNet()
        self.model.load_state_dict(state_dict)
        self.model.eval()
        
    def predict(self, obs, last_obs, index):
        x = self._make_input(obs, last_obs, index)
        with torch.no_grad():
            xt = torch.from_numpy(x).unsqueeze(0)
            p, v = self.model(xt)
            
        return p.squeeze(0).detach().numpy(), v.item()
        
    # Input for Neural Network
    def _make_input(self, obs, last_obs, index):
        b = np.zeros((17, 7 * 11), dtype=np.float32)
        
        for p, pos_list in enumerate(obs.geese):
            # head position
            for pos in pos_list[:1]:
                b[0 + (p - index) % 4, pos] = 1
            # tip position
            for pos in pos_list[-1:]:
                b[4 + (p - index) % 4, pos] = 1
            # whole position
            for pos in pos_list:
                b[8 + (p - index) % 4, pos] = 1

        # previous head position
        if last_obs is not None:
            for p, pos_list in enumerate(last_obs.geese):
                for pos in pos_list[:1]:
                    b[12 + (p - index) % 4, pos] = 1

        # food
        for pos in obs.food:
            b[16, pos] = 1

        return b.reshape(-1, 7, 11)

In [ ]:
%%writefile -a submission.py
game = HungryGeese()
state_dict = pickle.loads(bz2.decompress(base64.b64decode(PARAM)))
agent = NNAgent(state_dict)
mcts = MCTS(game, agent)

def alphageese_agent(obs, config):
#     print(obs)
    probs = mcts.getActionProb(obs, timelimit=config.actTimeout*1.)    
#     probs = mcts.getActionProb(obs, timelimit=0.05)

    action = game.actions[np.random.choice(np.argwhere(probs == probs.max()).flatten())]
    return action.name

In [ ]:
import torch
import base64
import bz2
import pickle
​
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
weights = torch.load('../input/hungry-geese-models/latest.pth', map_location=device)
PARAM = base64.b64encode(bz2.compress(pickle.dumps(weights)))

In [ ]:
# Read in the submission file
with open('submission.py',) as file:
    filedata = file.read()
​
# Replace the target string
filedata = filedata.replace('XXXXXXXXXX', PARAM.decode("utf-8") )
​
# Write the file out again
with open('submission.py','w') as file:
    file.write(filedata)